# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.weightstats import ztest

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
#Exploring the data
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

In [4]:
# number of callbacks for black-sounding names
callback_b= sum(data[data.race=='b'].call)
callback_b

157.0

In [5]:
# Total number of black-sounding name resumes
total_b= len(data[data.race=='b'])
total_b

2435

In [6]:
#Proportion of callback
prop_callback_b= (callback_b/total_b)
prop_callback_b

0.06447638603696099

In [7]:
# number of callbacks for white-sounding names
callback_w= sum(data[data.race=='w'].call)
callback_w

235.0

In [8]:
# Total number of black-sounding name resumes
total_w= len(data[data.race=='w'])
total_w

2435

In [9]:
#Proportion of callback
prop_callback_w= (callback_w/total_w)
prop_callback_w

0.09650924024640657

In [10]:
#Difference in proportion of call back
diff_in_callback= prop_callback_w - prop_callback_b
diff_in_callback

0.032032854209445585

### Question 1: What test is appropriate for this problem? Does CLT apply?

#####  Two Sample test is approprite to use on these datasets. And both t-test and z-test gives almost the same results since, the data is greater than 30. Also, the sampling of these data gives the normal distribution and thus holds CLT.

### Question 2: What are the null and alternate hypotheses?

##### Null Hypotheses: There is no difference between Black sounding and White sounding resumes or There is no difference in significant impact on the rate of callbacks for Black sounding and White sounding resumes. 

##### Alternate Hypotheses: There is a difference between Black sounding and White sounding resumes or There is a difference in significant impact on the rate of callbacks for Black sounding and White sounding resumes.

### Question 3: Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

#### i) Calculating Margin of error

In [11]:
#Standard of Error, SE= np.sqrt((p_hat-(1-p_hat))/n)
SE= np.sqrt((prop_callback_b*(1-prop_callback_b)/total_b)+ (prop_callback_w*(1-prop_callback_w))/total_w)
SE

0.0077833705866767544

In [12]:
#Margin of Error, ME
ME= SE*2
ME

0.015566741173353509

#### ii) Calculating Confidence interval

In [13]:
#Confidence interval, CI
CI= [diff_in_callback - ME, diff_in_callback + ME]
CI

[0.016466113036092078, 0.04759959538279909]

#### iii) Calculating p-value

In [14]:
#Computing white and black soung datasets
w = data[data.race=='w'].call
b = data[data.race=='b'].call

In [15]:
diff_in_mean= np.mean(w)-np.mean(b)

In [16]:
#Computing permutation samples
calls_concat= np.concatenate((w,b))
calls_reps= np.empty(10000)
for i in range (10000):
    calls_perm= np.random.choice(calls_concat, size=len(calls_concat))
    calls_reps[i]= np.mean(calls_perm[:len(w)])-np.mean(calls_perm[len(w):])

In [17]:
#Computing pvalue
p= np.sum(calls_reps>=diff_in_mean)/len(calls_reps)
p

0.0

In [18]:
#T-test Statistic
stats.ttest_ind(w, b)

Ttest_indResult(statistic=4.114705290861751, pvalue=3.940802103128886e-05)

### Question 4: Write a story describing the statistical significance in the context or the original problem.

##### According to the evidence of Confidence Interval and p-value, this analysis proves that there is a significant impact on interview callbacks with White sounding name and Black sounding name resumes. 

### Question 5: Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

##### Yes, as we see the analysis of Confidence Interval and Proportion of call back, there is more callbacks for White sounding name resumes than the Black sounding name resumes. However, this will not prove that the race is one of the factor in callback success since, there might be many other scenarios in the resumes to do tests and analysis. Also, there might be many other reasons for the call back success which we have not considered in the hypothesis testing.

In [22]:
np.mean(b)

0.0644763857126236

In [29]:
np.mean(w)

0.09650924056768417

In [34]:
d= data[data.call==1]

In [35]:
np.mean(d['race']=='w')

0.5994897959183674

In [33]:
d= data[data.race=='w']
np.mean(d['call']==1)

0.09650924024640657